In [1]:
import numpy as np
import pandas as pd
import os
from warnings import catch_warnings, simplefilter
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from math import sqrt

### 1.- CARGAREMOS EL DATASET DE ENTRADA DE LAS VALORACIONES PARA EL RECOMENDADOR Y LO DIVIDIREMOS EN DOS DATASETS DE TRAINING Y TEST USANDO ShuffleSplit en vez de Kfolds

In [2]:
def load_reviews_df(data_root='./Data/', reviews_file_name='beer_reviews_v3.csv',
                    columns=['user_id', 'beer_id', 'beeradvocate_score'], verbose=False):
    
    beer_reviews_file_name = os.path.join(data_root, reviews_file_name)
    if verbose:
        print("BEGIN load_reviews_df reviews LOADING ...\n")
        print("beer_reviews_file_name:", beer_reviews_file_name)
        print("columns:", columns)
    reviewsdfAux = pd.read_csv(beer_reviews_file_name, sep=',')
    reviewsdf = reviewsdfAux[columns].copy()
    return(reviewsdf)

as_matrix() ESTARA OBSOLETA EN EL FUTURO y se muestra un warning.

Aconsejan usar en su lugar una nueva funcion to_numpy() que llegara con pandas v0.24.0.

https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array

Como usamos as_matrix() anularemos ese WARNING que ya que nuestra version de pandas es anterior a v0.24.0. 

En el futuro habria que instalar pandas v0.24.0 y sustituir la llamada a as_matrix() por to_numpy().

In [3]:
def split_dataset(reviewsdf, X_columns_indexes, Y_column_indexes, nfolds=3, verbose=False):

    if verbose:
            print(X_columns_indexes)
            print(Y_column_indexes)
    
    with catch_warnings():
        simplefilter(action='ignore', category=FutureWarning)
        X = reviewsdf[X_columns_indexes].as_matrix()
        Y = reviewsdf[Y_column_indexes].as_matrix()
        
    kf = KFold(n_splits=3, random_state=15, shuffle=True)        
    if verbose:
        print("Splitting BEGIN -----------------")
        print(kf)
            
    c = 0
    for train_index, test_index in kf.split(X):
        if verbose and c < 20:
            c += 1
            print("Splitting Iteration Counter: {0}, TRAIN_index: {1}, TEST_index: {2}" \
                  .format(c, train_index, test_index))
        elif verbose:
            print("... and so on")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
            
    if verbose:
            print("Splitting END -------------------")
    
    return pd.DataFrame(X_train), pd.DataFrame(Y_train), pd.DataFrame(X_test), pd.DataFrame(Y_test)

In [3]:
cd "/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender"

/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender


In [4]:
beeradvocate_df = load_reviews_df("/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender/Data", verbose=True)

BEGIN load_reviews_df reviews LOADING ...

beer_reviews_file_name: /home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender/Data/beer_reviews_v3.csv
columns: ['user_id', 'beer_id', 'beeradvocate_score']


In [5]:
beeradvocate_df.head()

,user_id,beer_id,beeradvocate_score
0,0,0,4.37
1,1,0,4.24
2,2,0,4.37
3,3,0,4.03
4,4,0,3.29


In [6]:
beeradvocate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1301173 entries, 0 to 1301172
Data columns (total 3 columns):
user_id               1301173 non-null int64
beer_id               1301173 non-null int64
beeradvocate_score    1301173 non-null float64
dtypes: float64(1), int64(2)
memory usage: 29.8 MB


La division de TRAINING Y TEST la haremos con Kfolds y nfolds=3 ya que es el valor aconsejado para un DATASET GRANDE.

https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

In [8]:
X_train, Y_train, X_test, Y_test = split_dataset(beeradvocate_df, \
                                                 beeradvocate_df.columns[:2], beeradvocate_df.columns[2:3], \
                                                 10, verbose=False)

In [9]:
X_train.head()

,0,1
0,1,0
1,4,0
2,5,0
3,6,0
4,7,0


In [10]:
# nfolds=3, 867449 rows
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867449 entries, 0 to 867448
Data columns (total 2 columns):
0    867449 non-null int64
1    867449 non-null int64
dtypes: int64(2)
memory usage: 13.2 MB


In [11]:
X_test.head()

,0,1
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0


In [12]:
# nfolds=3, 433724 rows
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433724 entries, 0 to 433723
Data columns (total 2 columns):
0    433724 non-null int64
1    433724 non-null int64
dtypes: int64(2)
memory usage: 6.6 MB


No obstante, voy a descartar esta estrategia de division porque considero que deja fuera del DATASET DE TRAINING demasiados registros.

Voy a usar el metodo de BARAJEO Y DIVISION para controlar el numero de registros de TRAINING Y TEST y obtendre  1040938 registros de TRAINING y 260235 registros de TEST que me parece mejor division para el recomendador.

https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
"Random permutations cross-validation a.k.a. Shuffle & Split"

In [7]:
def shuffle_dataset(reviewsdf, X_columns_indexes, Y_column_indexes, \
                    n_splits=10, test_size=.20, train_size=0.80, random_state=0, verbose=False):

    columns=['user_id', 'beer_id', 'beeradvocate_score']
    
    if verbose:
            print(X_columns_indexes)
            print(Y_column_indexes)
    
    with catch_warnings():
        simplefilter(action='ignore', category=FutureWarning)
        X = reviewsdf[X_columns_indexes].as_matrix()
        Y = reviewsdf[Y_column_indexes].as_matrix()
        
    rs = ShuffleSplit(n_splits, test_size, train_size, random_state)
    if verbose:
        print("Shuffling BEGIN -----------------")
        print(rs)
            
    c = 0
    for train_index, test_index in rs.split(X):
        if verbose and c < 20:
            c += 1
            print("Shuffling Iteration Counter: {0}, TRAIN_index: {1}, TEST_index: {2}" \
                  .format(c, train_index, test_index))
        elif verbose:
            print("... and so on")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
    df_train = pd.concat( \
                [pd.DataFrame(X_train).reset_index().drop('index', axis=1), \
                 pd.DataFrame(Y_train).reset_index().drop('index', axis=1)], \
                 axis=1)
    df_train.columns = columns
                 
    df_test = pd.concat( \
                [pd.DataFrame(X_test).reset_index().drop('index', axis=1), \
                 pd.DataFrame(Y_test).reset_index().drop('index', axis=1)], \
                 axis=1)
    df_test.columns = columns
    
    if verbose:
            print("Shuffling END -------------------")
    
    return df_train, df_test

In [8]:
reviews_train, reviews_test = shuffle_dataset(beeradvocate_df, \
                                              beeradvocate_df.columns[:2], beeradvocate_df.columns[2:3], \
                                              10, verbose=True)

Index(['user_id', 'beer_id'], dtype='object')
Index(['beeradvocate_score'], dtype='object')
Shuffling BEGIN -----------------
ShuffleSplit(n_splits=10, random_state=0, test_size=0.2, train_size=0.8)
Shuffling Iteration Counter: 1, TRAIN_index: [ 947413  779311 1254976 ...  963395  117952  305711], TEST_index: [ 952522  789831  112280 ...  191401 1019561  195974]
Shuffling Iteration Counter: 2, TRAIN_index: [ 558245   37246 1119166 ...  406722  839194  918450], TEST_index: [ 281767  963376  639701 ...  575958 1271404   71611]
Shuffling Iteration Counter: 3, TRAIN_index: [ 396274  633467 1198197 ...  930239  798882 1142006], TEST_index: [ 583665 1080019 1099842 ...   38943  320954  184520]
Shuffling Iteration Counter: 4, TRAIN_index: [1152127  480365 1000074 ...  218213 1274826  582283], TEST_index: [1212540  405140 1290765 ...  502205 1071926  689458]
Shuffling Iteration Counter: 5, TRAIN_index: [ 656181   32820 1180048 ...  435386  676883  468681], TEST_index: [1059710  577766  861446 

In [9]:
reviews_train.head()

,user_id,beer_id,beeradvocate_score
0,4270,1387,2.74
1,285,514,4.32
2,4463,2777,3.13
3,2688,355,4.00
4,2670,223,4.32


In [10]:
reviews_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040938 entries, 0 to 1040937
Data columns (total 3 columns):
user_id               1040938 non-null int64
beer_id               1040938 non-null int64
beeradvocate_score    1040938 non-null float64
dtypes: float64(1), int64(2)
memory usage: 23.8 MB


In [11]:
reviews_test.head()

,user_id,beer_id,beeradvocate_score
0,8578,5279,4.65
1,1305,3129,4.12
2,1617,640,3.67
3,975,454,4.42
4,1802,6224,4.03


In [12]:
reviews_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260235 entries, 0 to 260234
Data columns (total 3 columns):
user_id               260235 non-null int64
beer_id               260235 non-null int64
beeradvocate_score    260235 non-null float64
dtypes: float64(1), int64(2)
memory usage: 6.0 MB


### 2.- CARGAREMOS LOS DATASETS DE USUARIOS, CERVEZAS Y ESTILOS DE CERVEZAS

In [13]:
def load_csv_to_dataframe(file_root='./Data/', file_name='beer_reviews_v3.csv', sep_char=',', verbose=False):
    
    csv_file_name = os.path.join(file_root, file_name)
    if verbose:
        print("BEGIN dataframe LOADING ...\n")
        print("csv_file_name:", csv_file_name, "\n")
    df = pd.read_csv(csv_file_name, sep=sep_char)
    if verbose:
        print("END dataframe LOADING ...\n")
    return(df)

In [14]:
users_df = load_csv_to_dataframe("/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender/Data", \
                                 "users_v1.csv", ',', verbose=False)

In [15]:
users_df.head()

,user_id,review_profilename
0,0,zyzygy
1,1,ypsifly
2,2,woemad
3,3,wnhay
4,4,williamherbert


In [16]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30810 entries, 0 to 30809
Data columns (total 2 columns):
user_id               30810 non-null int64
review_profilename    30810 non-null object
dtypes: int64(1), object(1)
memory usage: 481.5+ KB


In [17]:
beers_df = load_csv_to_dataframe("/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender/Data", \
                                 "beers_v1.csv", ',', verbose=False)

In [18]:
beers_df.head()

,beer_id,beer_name,beer_advocates_style,beer_abv,abv_strength,beer_style_ibu_avg,brewery_name
0,0,Stone Old Guardian Barley Wine Style Ale 2006,American Barleywine,11.20,4,80.0,Stone Brewing Co.
1,1,Samuel Adams Brown Ale,English Brown Ale,5.35,2,20.0,Boston Beer Company (Samuel Adams)
2,2,Hennepin (Farmhouse Saison),Belgian Saison,7.70,3,29.0,Brewery Ommegang
3,3,Double Bag,German Altbier,7.20,3,37.5,Long Trail Brewing Co.
4,4,Berkshire Russian Imperial Stout,Russian Imperial Stout,8.50,3,70.0,Berkshire Brewing Company Inc.


In [19]:
beers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6504 entries, 0 to 6503
Data columns (total 7 columns):
beer_id                 6504 non-null int64
beer_name               6504 non-null object
beer_advocates_style    6504 non-null object
beer_abv                6504 non-null float64
abv_strength            6504 non-null int64
beer_style_ibu_avg      6504 non-null float64
brewery_name            6504 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 355.8+ KB


In [20]:
styles_df = load_csv_to_dataframe("/home/dsc/Python_notebooks/TFM-ajao-Beer-Recommender/Data", \
                                 "beer_styles_v1.csv", ',', verbose=False)

In [21]:
styles_df.head()

,abv_range,abv_strength,family_name,ibu_range,ibu_strength,style_URL,style_id,style_name
0,6.3-7.6%,3,Bocks,20-30,3,https://www.beeradvocate.com/beer/styles/32/,0,German Bock
1,6.6-7.9%,3,Bocks,17-27,3,https://www.beeradvocate.com/beer/styles/35/,1,German Doppelbock
2,7.0-14.0%,4,Bocks,25-35,3,https://www.beeradvocate.com/beer/styles/36/,2,German Eisbock
3,6.3-8.1%,3,Bocks,20-38,3,https://www.beeradvocate.com/beer/styles/33/,3,German Maibock
4,7.0-9.5%,3,Bocks,15-35,3,https://www.beeradvocate.com/beer/styles/92/,4,German Weizenbock


In [22]:
styles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 8 columns):
abv_range       111 non-null object
abv_strength    111 non-null int64
family_name     111 non-null object
ibu_range       111 non-null object
ibu_strength    111 non-null int64
style_URL       111 non-null object
style_id        111 non-null int64
style_name      111 non-null object
dtypes: int64(3), object(5)
memory usage: 7.0+ KB


In [23]:
def get_beer_summary_by_id(beers_df, styles_df, beer_id):
    beer_summary_values = beers_df[beers_df['beer_id'] == beer_id][['beer_name', 'beer_advocates_style']].values.astype(str)[0]
    beer_summary = beer_summary_values[0] + ', ' + beer_summary_values[1]
    beer_style = styles_df[styles_df['style_name'] == beer_summary_values[1]]['family_name'].values.astype(str)[0]
    beer_summary +=  ' (' + beer_style + ')'
    return(beer_summary)

In [24]:
beer_id_list = range(0, 100, 10)
for beer_id in beer_id_list:
    print(get_beer_summary_by_id(beers_df, styles_df, beer_id))

Stone Old Guardian Barley Wine Style Ale 2006, American Barleywine (Strong Ales)
Mahr's Ungespundet-hefetrüb, German Kellerbier Zwickelbier (Pilseners and Pale Lagers)
Wexford Irish Cream Ale, American Cream Ale (Hybrid Beers)
Sierra Nevada Anniversary Ale (2007-2009), American IPA (India Pale Ales)
Samuel Adams Hallertau Imperial Pilsner, American Imperial Pilsner (Pilseners and Pale Lagers)
Odd Notion (Summer 08), English Dark Mild Ale (Brown Ales)
Leinenkugel's Red, Vienna Lager (Dark Lagers)
Hocus Pocus, American Pale Wheat Ale (Wheat Beers)
Chimay Première (Red), Belgian Dubbel (Dark Ales)
471 IPA, American Imperial IPA (India Pale Ales)


### 3.- METRICAS DE EVALUACION DE RECOMENDADORES Y DE SUS RECOMENDACIONES

In [25]:
class Metrics():
    """
    Implementa las dos metricas mas importantes para evaluar Sistemas de Recomendacion:
        . Recall
        . MAP
    """
    def __init__(self, train_ids, test_ids, recommended_ids):
        
        self.zipped = list(zip(test_ids, train_ids, recommended_ids))
        
    def recall_per_user(self, N, test, recommended, train):
        if train is not None: 
            rec_true = []
            for r in recommended:
                if r not in train:
                    rec_true.append(r)
            else:
                rec_true = recommended   
        intersection = len(set(test) & set(rec_true[:N]))
        return intersection / float(np.minimum(N, len(test)))
       
    def recall_at_n(self, topN):
        out = []
        for k in topN:
            recall = np.mean([self.recall_per_user(k, test, recom, train)  for (test, train, recom) in self.zipped])
            out.append(recall)
            print("recall@%s=%.3f" %(k, recall))
        return out
                                       
    def apk(self, N, test, recommended, train):
        if train is not None: 
            rec_true = []
            for r in recommended:
                if r not in train:
                    rec_true.append(r)
        else:
            rec_true = recommended    
        predicted = rec_true[:N] # top-k predictions

        score = 0.0 # This will store the numerator
        num_hits = 0.0 # This will store the sum of rel(i)

        for i,p in enumerate(predicted):
            if p in test and p not in predicted[:i]:
                num_hits += 1.0
                score += num_hits/(i+1.0)

        return score / min(len(test), N)

    def map_at_n(self, topN):
        out = []
        for k in topN:
            map_ = np.mean([self.apk(k, test, recom, train)  for (test, train, recom) in self.zipped])
            out.append(map_)
            print("map@%s=%.3f" %(k, map_))
        return out

### 4.- RECOMENDADORES BASADOS EN LA POPULARIDAD (BASELINE)

In [25]:
class popularity_models(object):
    """
    Se implementan los siguientes recomendadores basados en medidas de popularidad (model):
        . mostRated: La cerveza que se ha valorado mayor numero de veces (se asumira que es la cerveza que nas se ha bebido)
        . positiveRated: La cerveza que tiene valoraciones positivas por encima de un valor limite dado (threshold)
        . mean_rate_movies: La cerveza que tiene la valoracion mas alta de media y tiene un numero minimo de valoraciones
    """
    def __init__(self, model, train_df):
        self.model_name = model
        self.train_df = train_df
    
    def mostRated(self):
        """
        Recomendador 'La cerveza que se ha valorado mayor numero de veces'
        """
        return self.train_df.groupby('beer_id')['user_id'].count().sort_values(ascending=False)

    def positiveRated(self, min_score=4.0):
        """
        Recomendador 'La cerveza que tiene valoraciones positivas por encima de un valor':
          beeradvocate_score > min_score
        """
        return self.train_df[self.train_df['beeradvocate_score'] >= min_score].groupby('beer_id')['user_id'].count().sort_values(ascending=False) 
        
    def mean_rate_beers(self, min_scores=50):
        """
        Recomendador 'La cerveza que tiene la valoracion mas alta de media':
            Solo se tendran en cuenta ccervezas con mayor numero de valoraciones que min_scores
        """
        listRatedBeers = self.train_df.groupby('beer_id')['beeradvocate_score'].apply(list).reset_index()
        filteredListRatedBeers = listRatedBeers[listRatedBeers['beeradvocate_score'].apply(lambda x: len(x) > min_scores)]
        meanBeers = filteredListRatedBeers['beeradvocate_score'].apply(lambda x: np.mean(np.array(x))).sort_values(ascending=False)
        return meanBeers
        
    def train(self, min_rating=4.0, min_ratings=50):
        """
        Entrenamiento del Recomendador de Popularidad En Uso (Instanciado)
        """
        if self.model_name == 'mostRated':
            self.model = self.mostRated()
        elif self.model_name == 'positiveRated':
            self.model = self.positiveRated(min_rating)
        elif self.model_name == 'mean_rate_movies':
            self.model = self.mean_rate_movies(min_ratings)
        else:
            raise ValueError('%s no existe' % self.model_name)
        
    def predict(self, top_k, verbose=False):
        """
        Devuelve las primeras top_k recomendaciones del Recomendador de Popularidad En Uso
        """
        return self.model.index.values[:top_k]
    
    def get_recommended_beers(self, beers_ids, beers_df):
        """
        Dada una lista de ids de cervezas, devuelve una lista la informacion resumida de cada una
        """
        beers_stuff = []
        for i, index in enumerate(beers_ids):
            id_ = index # id de la cerveza
            beers_stuff.append(str(id_) + ' ' + get_beer_summary_by_id(beers_df, styles_df, id_))

        return(list(beers_stuff))

#### 4.1.- SISTEMA DE RECOMENDACION BASADO EN POPULARIDAD MostRated

In [26]:
sr_mostRated = popularity_models('mostRated', reviews_train)
sr_mostRated.train()
sr_mostRated.model.head()

beer_id
223    2508
142    2421
12     2311
179    2093
130    2067
Name: user_id, dtype: int64

In [27]:
sr_mostRated.predict(10)

array([223, 142,  12, 179, 130, 250, 136, 134, 312, 294])

In [28]:
for recommended_beer in sr_mostRated.get_recommended_beers(sr_mostRated.predict(10), beers_df):
    print(recommended_beer)

223 90 Minute IPA, American Imperial IPA (India Pale Ales)
142 Old Rasputin Russian Imperial Stout, Russian Imperial Stout (Stouts)
12 Sierra Nevada Celebration Ale, American IPA (India Pale Ales)
179 Arrogant Bastard Ale, American Strong Ale (Strong Ales)
130 Stone Ruination IPA, American Imperial IPA (India Pale Ales)
250 Two Hearted Ale, American IPA (India Pale Ales)
136 Sierra Nevada Pale Ale, American Pale Ale (APA) (Pale Ales)
134 Stone IPA (India Pale Ale), American IPA (India Pale Ales)
312 Sierra Nevada Bigfoot Barleywine Style Ale, American Barleywine (Strong Ales)
294 Brooklyn Black Chocolate Stout, Russian Imperial Stout (Stouts)


EVALUACION DEL SISTEMA DE RECOMENDACION MODELO MostRated

In [37]:
topk = [5, 10, 20]
trainUsersGrouped = reviews_train.groupby('user_id')['beer_id'].apply(list).reset_index().sort_index()
train_ids = trainUsersGrouped['beer_id'].values
testUsersGrouped = reviews_test.groupby('user_id')['beer_id'].apply(list).reset_index().sort_index()
test_ids = testUsersGrouped['beer_id'].values

predictions = trainUsersGrouped['beer_id'].apply(lambda x: sr_mostRated.predict(np.max(topk))).reset_index()
predictions_ids = predictions['beer_id'].values
m = Metrics(train_ids, test_ids, predictions_ids)
print('-----MEDIDA Recall-----')
_ = m.recall_at_n(topk)
print('-----MEDIDA Map-----')
_ = m.map_at_n(topk)

-----MEDIDA Recall-----
recall@5=0.042
recall@10=0.051
recall@20=0.077
-----MEDIDA Map-----
map@5=0.021
map@10=0.018
map@20=0.017


AHORA EVALUAREMOS TODOS LOS SISTEMAS DE RECOMENDACION BASADOS EN POPULARIDAD.

##### Los resultados son bajos para todos pero el que consigue mejores resultados es mostRated.

In [57]:
# Dataframe['user_id', Lista_'beer:id'] que contiene cada user_id y una lista de los beer_id que ha valorado
trainUsersGrouped = reviews_train.groupby('user_id')['beer_id'].apply(list).reset_index().sort_index()
train_ids = trainUsersGrouped['beer_id'].values
testUsersGrouped = reviews_test.groupby('user_id')['beer_id'].apply(list).reset_index().sort_index()
test_ids = testUsersGrouped['beer_id'].values

In [38]:
rs_popularity_list = ['mostRated', 'positiveRated', 'mean_rate_beers']
topk = [5, 10, 20, 50]
for rs_model_name in rs_popularity_list:
    rs_popularity = popularity_models(rs_model_name, reviews_train)
    rs_popularity.train()
    
    predictions = trainUsersGrouped['beer_id'].apply(lambda x: rs_popularity.predict(np.max(topk))).reset_index()
    predictions_ids = predictions['beer_id'].values
    m = Metrics(train_ids, test_ids, predictions_ids)
    print('-----------------------')
    print(rs_model_name)
    print('-----MEDIDA Recall-----')
    _ = m.recall_at_n(topk)
    print('-----MEDIDA Map-----')
    _ = m.map_at_n(topk)
    print('-----------------------\n')


-----------------------
mostRated
-----MEDIDA Recall-----
recall@5=0.042
recall@10=0.051
recall@20=0.077
recall@50=0.134
-----MEDIDA Map-----
map@5=0.021
map@10=0.018
map@20=0.018
map@50=0.018
-----------------------

-----------------------
positiveRated
-----MEDIDA Recall-----
recall@5=0.040
recall@10=0.053
recall@20=0.076
recall@50=0.126
-----MEDIDA Map-----
map@5=0.020
map@10=0.018
map@20=0.017
map@50=0.017
-----------------------

-----------------------
mean_rate_movies
-----MEDIDA Recall-----
recall@5=0.003
recall@10=0.005
recall@20=0.013
recall@50=0.026
-----MEDIDA Map-----
map@5=0.001
map@10=0.001
map@20=0.002
map@50=0.002
-----------------------



### 5.- RECOMENDADOR BASADO EN MATRIZ DE CO-OCURRENCIA

LAS RECOMENDACIONES SE BASAN EN EL GRADO DE SIMILITUD QUIE TIENEN ENTRE SI LOS CERVEZAS.

LA RECOMENDACION DE CERVEZA PARA UN USUARIO CONCRETO SE HARA ASI:

    1.- Una funcion devolvera los topN cervezas mas similares a una dada:
co_occurrance_similarity(item_id, coocurrance, ntop=10) 

    2. Otra funcion co_occurrance_recommendation(items_id, cooccurrance, ntop=10),
para un usuario dado y la lista de cervezas que ha valorado, 
seleccionaremos las topN cervezas de las listas de cervezas mas similares 
que correspondan a cada cerveza que ha valorado ese usuario. 

In [13]:
def get_co_occurrance_matrix(min_rating=4):
    
    # LOS PARAMETROS DE VALORACION DE CERVEZAS POR ENCIMA DE min_rating
    # CREAREMOS UN DICCIONARIO { 'user_id': array numpy['beer_id'_1, .., 'beer_id'_n], ...}
    beersPerUser = (reviews_train[reviews_train['beeradvocate_score'] >= min_rating]
                 .groupby('user_id')['beer_id']
                 .apply(np.array)
                 .to_dict()
                )
    
    # CREAREMOS LA MATRIZ DE CO-OCURRENCIA
    num_users = users_df['user_id'].max() + 1
    num_beers = beers_df['beer_id'].max() + 1
    coMatrix = np.zeros((num_users, num_beers))
    for beers in beersPerUser.values():
        for b in beers:
            coMatrix[b, beers] += 1
    return coMatrix, beersPerUser

LOS PARAMETROS DE VALORACION DE CERVEZAS POR ENCIMA DE 3.50 COMIENZAN EL TRAMO DE LAS QUE FUERON CONSIDERADAS BUENAS CERVEZAS POR LOS USUARIOS: 

son las que interesan para recomendar otras buenas cervezas qu sean similares a ellas

In [14]:
coMatrix, beersPerUser = get_co_occurrance_matrix(3.5)

In [15]:
def co_occurrance_similarity(item_id, coocurrance, ntop=10):
    """
    Devuelve las cervezas topN mas similares a una dada usando la matriz de co-ocurrencia
    
    :param item_id: id de la cerveza
    :param cooccurrance: matriz de co-occurencia (numpy array)
    :param ntop: numero de cervezas mas similares a la dada como item,_id que se van a devolver    
    """
    similarItems = coocurrance[item_id, :]
    # Devuelve indices de los items mas similares en orden descendente
    mostSimilar = np.argsort(similarItems)[::-1]
    # se quita el primer elemento que es la propia cerveza para la que busco las topN similares
    mostSimilar = mostSimilar[1:ntop+1]
    
    # Puntuacion o Grado de Similaridad de cada item_id o cerveza mas similar
    # similarItems[mostSimilar])
    
    # Devuelve un array numpy con indice (primera columna = item_id = beer_id) y,
    # el Grado de Similaridad de esos items o cervezas mas similares (segunda columna = similaridad)
    return np.stack((mostSimilar, similarItems[mostSimilar])).T
    # Es mejor pero tambien valdria:
    #return np.stack((mostSimilar, similarItems[mostSimilar]), axis=1)

In [16]:
# Obtengamos las 5 cervezas mas similares a de la de beer_id = 49
print("La Cerveza id = {0}, '{1}', tiene esta matriz de co-ocurrencia:"\
      .format(49, beers_df[beers_df['beer_id'] == 49]['beer_name'].values.astype(str)[0]))
print(co_occurrance_similarity(49, coMatrix, 5))

La Cerveza id = 49, 'Old Chub Scottish Style Ale', tiene esta matriz de co-ocurrencia:
[[142. 452.]
 [223. 424.]
 [ 12. 401.]
 [294. 400.]
 [130. 399.]]


In [17]:
# Otros ejemplos
queryBeerIds = [49, 70, 300]
Ntop = 5
for queryBeerId in queryBeerIds:
    print('Para la cerveza de id = "%s", "%s", \nlas top-%s recommendaciones son:' % \
          (queryBeerId, \
           get_beer_summary_by_id(beers_df, styles_df, queryBeerId), \
           Ntop))

    similarBeers = co_occurrance_similarity(queryBeerId, coMatrix, Ntop)
    for similarBeer in similarBeers:
        print(int(similarBeer[0]), \
              get_beer_summary_by_id(beers_df, styles_df, similarBeer[0]), \
              similarBeer[1])
    print("\n")

Para la cerveza de id = "49", "Old Chub Scottish Style Ale, Scotch Ale Wee Heavy (Strong Ales)", 
las top-5 recommendaciones son:
142 Old Rasputin Russian Imperial Stout, Russian Imperial Stout (Stouts) 452.0
223 90 Minute IPA, American Imperial IPA (India Pale Ales) 424.0
12 Sierra Nevada Celebration Ale, American IPA (India Pale Ales) 401.0
294 Brooklyn Black Chocolate Stout, Russian Imperial Stout (Stouts) 400.0
130 Stone Ruination IPA, American Imperial IPA (India Pale Ales) 399.0


Para la cerveza de id = "70", "Hocus Pocus, American Pale Wheat Ale (Wheat Beers)", 
las top-5 recommendaciones son:
351 Storm King Stout, Russian Imperial Stout (Stouts) 54.0
294 Brooklyn Black Chocolate Stout, Russian Imperial Stout (Stouts) 51.0
179 Arrogant Bastard Ale, American Strong Ale (Strong Ales) 48.0
12 Sierra Nevada Celebration Ale, American IPA (India Pale Ales) 48.0
136 Sierra Nevada Pale Ale, American Pale Ale (APA) (Pale Ales) 48.0


Para la cerveza de id = "300", "Sierra Nevada Tumbler

En las cervezas similares se observa que las cervezas mas populares, las que tienen mas votos y eran recomendadas por el recomendador mostRated, producen cierto sesgo en las recomendaciones de la matriz de coocurrencia ya que aparecen varias de ellas entre las primeras recomendaciones:

223 90 Minute IPA, American Imperial IPA (India Pale Ales)

142 Old Rasputin Russian Imperial Stout, Russian Imperial Stout (Stouts)

12 Sierra Nevada Celebration Ale, American IPA (India Pale Ales)

179 Arrogant Bastard Ale, American Strong Ale (Strong Ales)

130 Stone Ruination IPA, American Imperial IPA (India Pale Ales)

250 Two Hearted Ale, American IPA (India Pale Ales)

136 Sierra Nevada Pale Ale, American Pale Ale (APA) (Pale Ales)

134 Stone IPA (India Pale Ale), American IPA (India Pale Ales)

312 Sierra Nevada Bigfoot Barleywine Style Ale, American Barleywine (Strong Ales)

294 Brooklyn Black Chocolate Stout, Russian Imperial Stout (Stouts)

In [18]:
# AHORA LA FUNCION PARA PODER HACER LAS RECOMENDACIONES
def co_occurrance_recommendation(items_id, cooccurrance, ntop=10):
    """
    Devuelve la lista de las ntop recomendaciones en base a la lista de items o cervezas 
    que ha valorado el usuario (historia de valoraciones del usuario)
    
    :param items_id: lista de items ids de las cervezas valoradas por el usuario
    :param coocurrence: matriz de co-ocurrencia (array 2d numpy)
    :param ntop: top-K items o cervezas que se devolveran    
    """

    # Obtendremos las listas de items similares de cada item que ha valorado el usuario
    # y guardamos cada lista en una fila
    list_sim_items = np.vstack([co_occurrance_similarity(id_, cooccurrance, ntop) for id_ in items_id])
    
    # Agrupamos por cada beer_id y borramos duplicados teniendo en cuenta su frecuencia
    largest_freq = pd.DataFrame(list_sim_items, columns=['id', 'freq']).groupby('id').agg(max).reset_index()
    
    # ordenamos por grado de similaridad en orden descendente
    sorted_list = largest_freq.sort_values(by='freq', ascending=False)
    
    # obtenemos los ntop
    out = sorted_list.values[:ntop, 0]
    return out

In [21]:
Ntop = 50
predictions = trainUsersGrouped['beer_id'].apply(lambda x: 
    co_occurrance_recommendation(x, coMatrix, Ntop)).reset_index()\
    .rename(columns={"beer_id":"similar_beer_ids", "index":"beer_id"})
# index = beer_id que ha valorado un user_id dado, 
# beer_id = List of item_ids mas similares o Recommendations
predictions.head()

,beer_id,similar_beer_ids
0,0,"[1063.0, 294.0, 223.0, 142.0, 42.0, 136.0, 312..."
1,1,"[142.0, 130.0, 134.0, 179.0, 225.0, 223.0, 12...."
2,2,"[223.0, 142.0, 130.0, 179.0, 134.0, 12.0, 312...."
3,3,"[223.0, 142.0, 130.0, 179.0, 134.0, 312.0, 12...."
4,4,"[223.0, 142.0, 130.0, 179.0, 134.0, 12.0, 312...."


In [22]:
queryBeerIds = predictions.head()['beer_id']
for queryBeerId in queryBeerIds:
    print('Para la cerveza de id = "%s", "%s", \nlas top-%s recommendaciones son:' % \
          (queryBeerId, \
           get_beer_summary_by_id(beers_df, styles_df, queryBeerId), \
           Ntop))

    similarBeers = predictions[predictions['beer_id'] == queryBeerId]['similar_beer_ids'].tolist()[0]
    counter = 0
    for similarBeer in similarBeers:
        counter += 1
        print(int(similarBeer), \
              get_beer_summary_by_id(beers_df, styles_df, int(similarBeer)))
        if counter == 10:
            print('.... y hasta Ntop = 50 mas')
            break
    print("\n")

Para la cerveza de id = "0", "Stone Old Guardian Barley Wine Style Ale 2006, American Barleywine (Strong Ales)", 
las top-50 recommendaciones son:
1063 Samuel Adams Black Lager, German Schwarzbier (Dark Lagers)
294 Brooklyn Black Chocolate Stout, Russian Imperial Stout (Stouts)
223 90 Minute IPA, American Imperial IPA (India Pale Ales)
142 Old Rasputin Russian Imperial Stout, Russian Imperial Stout (Stouts)
42 Samuel Adams Boston Lager, Vienna Lager (Dark Lagers)
136 Sierra Nevada Pale Ale, American Pale Ale (APA) (Pale Ales)
312 Sierra Nevada Bigfoot Barleywine Style Ale, American Barleywine (Strong Ales)
12 Sierra Nevada Celebration Ale, American IPA (India Pale Ales)
179 Arrogant Bastard Ale, American Strong Ale (Strong Ales)
130 Stone Ruination IPA, American Imperial IPA (India Pale Ales)
.... y hasta Ntop = 50 mas


Para la cerveza de id = "1", "Samuel Adams Brown Ale, English Brown Ale (Brown Ales)", 
las top-50 recommendaciones son:
142 Old Rasputin Russian Imperial Stout, Russi

En las cervezas recomendadas tambien se observa que las cervezas mas populares, las que tienen mas votos y eran recomendadas por el recomendador mostRated, producen cierto sesgo en las recomendaciones de la matriz de coocurrencia aunque ahora al menos se recomienda una cerveza distinta de ellas entre las 10 primeras.

In [23]:
# VAMOS A EVALUAR EL RECOMENDADOR POR MATRIZ DE CO-OCURRENCIA
topk = [5, 10, 20, 50]
predictions_ids = predictions['similar_beer_ids'].values
m = Metrics(train_ids, test_ids, predictions_ids)
print('-----MEDIDA Recall-----')
_ = m.recall_at_n(topk)
print('-----MEDIDA Map-----')
_ = m.map_at_n(topk)

-----MEDIDA Recall-----
recall@5=0.038
recall@10=0.050
recall@20=0.069
recall@50=0.123
-----MEDIDA Map-----
map@5=0.019
map@10=0.016
map@20=0.015
map@50=0.015


recall@n de la Matriz de Co-ocurrencia es un poco mas baja que mostRated, muy similar.

map@n igualmente tambien es un poco mas baja.

### 6.- RECOMENDADOR DE FILTRADO COLABORATIVO TIPO ALS (Alternating Least Squares)

Ver:
https://medium.com/datos-y-ciencia/intro-als-pyspark-7de7f3ba3b0a

Es un algoritmo de factorización matricial con regularización que evita el sobreajuste del algoritmo SVD (Singular Value Decomposition) al aumentar su numero de dimensiones.

The ALS algorithm aims to estimate two unknown matrices which, when multiplied together, yield the rating matrix.

ALS se usa para calcular las matrices q_i y p_u mediante factorización de la matriz de interacciones usuarios-ítems con pesos que si se multiplican da como resultado la matriz de valoraciones estimada r_ui_estimada. 

La funcion de perdida (loss) aplicada que se optimizara con ALS es la suma de errores al cuadrado:

<img src="https://latex.codecogs.com/gif.latex?\underset{Q*&space;,&space;P*}{min}\sum_{(u,i)\epsilon&space;K&space;}(r_{ui}-P_u^TQ_i)^2&plus;\lambda(\left&space;\|&space;Q_i&space;\right&space;\|^2&space;&plus;&space;\left&space;\|&space;P_u&space;\right&space;\|^2)$&space;&space;$" title="\underset{q* , p*}{min}\sum_{(u,i)\epsilon K }(r_{ui}-q_i^Tp_u)^2+\lambda(\left \| q_i \right \|^2 + \left \| p_u \right \|^2)" />

<img alt="factorization" src=https://miro.medium.com/max/1421/1*SVgAXmUZjBal1-NXAoB_fA.jpeg
style="width: 600px"/>
<br clear="all"/>

ALS se basa en el hecho de que la optimización se aborda como dos problemas cuadráticos obtenidos fijando, de forma alternativa, los términos p_u y q_i, en principio desconocidos. Cuando todos los términos p_u han sido fijados, el sistema hace iteraciones para optimizar los términos q_i por mínimos cuadrados y despues realizara iteraciones fijando los términos q_i para optimizar los terminos p_u [Koren, Y., Bell, R., Volinsky, C., 2009]

In [26]:
def alsRmse(I,R,P,Q):
    return np.sqrt(np.sum((I * (R - np.dot(P,Q.T)))**2)/len(R[R > 0]))

In [27]:
import time
def iteration(user, fixed_vecs, counts, num_factors, reg_param, num_solve, verbose=False):
    num_fixed = fixed_vecs.shape[0]
    YTY = fixed_vecs.T.dot(fixed_vecs)
    eye = np.eye(num_fixed)
    lambda_eye = reg_param * np.eye(num_factors)
    solve_vecs = np.zeros((num_solve, num_factors))

    t = time.time()
    for i in range(num_solve):
        if user:
            counts_i = counts[i]
        else:
            counts_i = counts[:, i].T
        CuI = np.diag(counts_i)
        pu = counts_i.copy()
        pu[np.where(pu != 0)] = 1.0
        YTCuIY = fixed_vecs.T.dot(CuI).dot(fixed_vecs)
        YTCupu = fixed_vecs.T.dot(CuI + eye).dot(pu.T)
        xu = np.linalg.solve(YTY + YTCuIY + lambda_eye, YTCupu)
        solve_vecs[i] = xu
        if verbose and i % 300 == 0:
            print('Solved %i vecs in %d seconds' % (i, time.time() - t))
            t = time.time()

    return solve_vecs

In [28]:
# COMPROBAMOS EL RENDIMIENTO DE ALS DIBUJANDO UNA GRAFICA CON LOS ERRORES ENTRAIN Y EN TEST
import matplotlib.pyplot as plt
%matplotlib inline

def check_als_performance(n_epochs, train_errors, test_errors):
    plt.plot(range(n_epochs), train_errors, marker='o', label='Training Data');
    plt.plot(range(n_epochs), test_errors, marker='v', label='Test Data');
    plt.title('ALS-WR Learning Curve')
    plt.xlabel('Number of Epochs');
    plt.ylabel('RMSE');
    plt.ylim(1, 5)
    plt.legend()
    plt.grid()
    plt.show()

In [26]:
def get_r_ui_matrix(dfratings, training_flag=True, verbose=False):
    # dfratings dataset de ratings de TRAINING o de TEST
    # El valor de 'user_id' y 'beer_id' empiezan en cero
    uMatrixTraining = np.zeros((users_df['user_id'].max()+1, \
                                beers_df['beer_id'].max()+1))
    if training_flag:
        print('INICIO Generacion Matriz r_ui de TRAINING')
    else:
        print('INICIO Generacion Matriz r_ui de TEST')
    for row in dfratings.values[:,0:3]:
        # Note user ids start at 1
        user = int(row[0]) # row['user_id']
        beer = int(row[1]) # row['beer_id']
        rating = row[2] # row['beeradvocate_score']
        if verbose:
            print(user, beer, rating)
        uMatrixTraining[user, beer] = rating

    print('FINAL Generacion Matriz r_ui de TRAINING')

    return uMatrixTraining

## SI DEFINO EL RECOMENDADOR ALS en un metodo train_als_recommender me da error Memory Error.

def train_als_recommender(r_ui_Training, r_ui_Test, num_iterations, num_factors, lambda_term, verbose)
    ...
    return user_vectors, item_vectors, train_errors, test_errors

Voy a probar si meto todo el codigo en un solo bloque y solo cargolas variables que necesito para ALS a ver si no vuelve a fallar por Memory Error.

In [27]:
r_ui_MatrixTraining = get_r_ui_matrix(reviews_train, True, verbose=False)

INICIO Generacion Matriz r_ui de TRAINING
FINAL Generacion Matriz r_ui de TRAINING


In [28]:
print(r_ui_MatrixTraining.shape)
users_df['user_id'].max()+1, beers_df['beer_id'].max()+1

(30810, 6504)


(30810, 6504)

In [29]:
r_ui_MatrixTesting = get_r_ui_matrix(reviews_test, False, verbose=False)

INICIO Generacion Matriz r_ui de TEST
FINAL Generacion Matriz r_ui de TRAINING


In [30]:
print(r_ui_MatrixTesting.shape)
users_df['user_id'].max()+1, beers_df['beer_id'].max()+1 

(30810, 6504)


(30810, 6504)

#### Intento entrenar un ALS para 30.810 user_ids x 6.504 bber_ids = 200.388.240, A LO MEJOR NECESITO PySpark para este modelo y estos DATASET de Ratings r_uid

In [37]:
#def train_als_recommender(r_ui_Training, r_ui_Test, \
#                          num_iterations=10, num_factors=50, lambda_term=0.2, verbose=True):
    # r_ui_MatrixTraining matriz de ratings (user_id x beer_id) de TRAINING
    # r_ui_MatrixTesting matriz de ratings (user_id x beer_id) de TEST
    # num_iterations ES NUMERO DE ITERACIONES DE CALCULO DE OPTIMIZACIONES
    # num_factors ES EL NUMERO DE FACTORES DE LAS MATRICES A CALCULAR p_u y q_i
    # lambda_term ES EL TERMINO DE REGUILARIZACION PARA EVITAR EL OVERFITTING
num_iterations=10
#num_factors=50 <== HAY Memory Error
num_factors=20
lambda_term=0.2

num_users = users_df['user_id'].max() + 1
num_beers = beers_df['beer_id'].max() + 1
num_pairs = reviews_train.shape[0]
R = r_ui_MatrixTraining.copy()
p = r_ui_MatrixTraining.copy()
p[p > 0] = 1
p[p == 0] = 0
T = r_ui_MatrixTesting.copy()
ptest = r_ui_MatrixTesting.copy()
ptest[ptest > 0] = 1
ptest[ptest == 0] = 0
alpha = num_pairs / (num_users * num_beers)
C = alpha * R
user_vectors = np.random.normal(size=(num_users, num_factors))
item_vectors = np.random.normal(size=(num_beers, num_factors))
train_errors = []
print("INICIO DE {0} INTERACIONES, {1} FACTORES Y REGULARIZACION LAMBDA {2}."\
    .format(num_iterations, num_factors, lambda_term))
test_errors = []
for i in range(num_iterations):
    print('Calculando optimizaciones para user_vectors...')
    user_vectors = iteration(True, item_vectors, C, num_factors, lambda_term, num_users)
    print('Calculando optimizaciones para item_vectors...')
    item_vectors = iteration(False, user_vectors, C, num_factors, lambda_term, num_beers)
    print('iteracion %i finalizada' % (i + 1))

    train_rmse = alsRmse(p, r_ui_MatrixTraining, user_vectors, item_vectors)
    test_rmse = alsRmse(ptest, r_ui_MatrixTesting, user_vectors, item_vectors)
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
    print("[Iteracion %d de %d] train error: %f, test error: %f" 
        %(i+1, num_iterations, train_rmse, test_rmse))

print("FINAL DE LAS INTERACIONES")
    
    #return user_vectors, item_vectors, train_errors, test_errors

MemoryError: 

### 7.- RECOMENDADOR DE FILTRADO COLABORATIVO TIPO SVD (Singular Value Decomposition)

En los modelos de Filtrado Colaborativo de Factorizacion de Matrices se intentan obtener los factores latentes (vectores) para poder modelizar las interacciones de los usuarios con los items que, a diferencia del la matriz de coocurrencia, los espaciones latentes son de menor tamaño (alrededor de 100 dimensiones) y se pueden usar para hacer las recomendaciones.

## $$r_{u,i} \approx {\bf f}_u^T\cdot{\bf f}_i$$

En el caso de SVD el objetivo es reducir la dimensionalidad del espacio de entrada: SVD es casi lo mismo que la Descomposicion Eigen o PCA (Principal Component Analysis).

METRICA RMSE

In [32]:
def rmse(prediction, ground_truth):
    """
    Return the Root Mean Squared Error of the prediction
    
    :param prediction: Un array numpy de 2d con las recomendaciones (predicciones a aevaluar)
    :param ground_truth: Un array numpy de 2d con valoraciones reales (del dataset TEST)
    
    :return the RMSE
    """
    # Obtiene los indices de los elementos que no son cero dentro del dataset TEST
    r, c = ground_truth.nonzero()
    # Obtiene los elementos que no son cero dentro de las recomendaciones
    p = prediction[r,c]
    # Obtiene los elementos que no son cero dentro del dataset TEST
    t = ground_truth[r,c]
    return sqrt(np.mean(np.power(p-t, 2.0)))

In [31]:
from scipy.sparse.linalg import svds

In [34]:
k=20
u, s, vt = svds(r_ui_MatrixTesting, k)

Comprobaremos que las matrices generadas son ortogonales y tienen los tamaños adecuados

In [35]:
# U tiene 30810 user_ids x 20 = k factores, dimensiones correctas
print(u.shape)
# U es ortogonal
print(rmse(np.dot(u.T,u), np.identity(k)))

(30810, 20)
1.0453208183364293e-15


In [36]:
# V tiene 20 = k factores x 6504 beers_ids, dimensiones correctas
print(vt.shape)
# V es ortogonal
print(rmse(np.dot(vt,vt.T), np.identity(k)))

(20, 6504)
1.0979426536895535e-15


##### Calculo de las recomendaciones

Calcularemos la matriz M (r_uid) de valoraciones (user_idx x beers_ids) segun esta formula:

### $$M\approx U\mathrm{diag}(s)V^T$$

El primer paso será crear la matriz diagonal de s.

In [37]:
s_diag_matrix = np.diag(s)

Calcularemos las recomendaciones o predicciones segun la formula anterior.

In [38]:
svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)

Comprobamos que las dimensiones son correctas comparandolas con la matriz r_ui de Testing

In [39]:
print(svdPredictions.shape)
print(r_ui_MatrixTesting.shape)

(30810, 6504)
(30810, 6504)


#### EVALUAREMOS LAS PREDICCIONES Y EL MODELO SVD CON 20 DIMENSIONES LATENTES

In [40]:
print('SVD RMSE=%.3f' % rmse(svdPredictions, r_ui_MatrixTesting))

SVD RMSE=3.550


In [58]:
# recall para las primeras 200 predicciones de cada user_id
svdPredictions_sorted = np.argsort(svdPredictions)[::-1]
svdPredictions_sorted = svdPredictions_sorted[:, :200]
test_ids = testUsersGrouped['beer_id'].values
predicted_ids = svdPredictions_sorted
topk = [5, 10, 20, 50, 100]
m = Metrics(train_ids, test_ids, predicted_ids)
print('*****Recall*****')
_ = m.recall_at_n(topk)
print('*****Map*****')
_ = m.map_at_n(topk)

*****Recall*****
recall@5=0.012
recall@10=0.015
recall@20=0.019
recall@50=0.029
recall@100=0.040
*****Map*****
map@5=0.006
map@10=0.005
map@20=0.004
map@50=0.004
map@100=0.004


## NOTA: Este es el tercer recomendador que he podido implementar porque la siguiente variante de SVD dió el mismo problema de memoria que ALS como se puede ver más abajo.

### Feedback Implicito vs Feedback Explicito

El anterior algoritmo SVD intentan calcular o estimar la matriz de valoraciones (ratings).

Sin embargo, es mas facil intentar modelizar la matriz de preferencias, o sea, si una cerveza le gusta o no a un  usuario.

Necsitamos crear una matriz "selector" $I$ para la matriz de r_ui de valoraciones $R$ de forma que contenga 0 si no existia valoracion y 1 si existia valoracion para esa entrada y, se hara lo mismo para los datos de test.

Convertiremos uMatrixTraining and uMatrixTesting en datos implicitos (I e I2 respectivamente) de forma que en estas nuesvas matrices si la valoracion es mayor que 3.5 (buena cerveza) entonces rellenaremos con 1 y si no rellenaremos con 0.


In [59]:
# Matriz Indice para los datos de training
I = r_ui_MatrixTraining.copy()
I[I > 3.50] = 1
I[I == 0] = 0

In [60]:
# Matriz Indice para los datos de test
I2 = r_ui_MatrixTesting.copy()
I2[I2 > 3.50] = 1
I2[I2 == 0] = 0

Entrenamos SVD con estas valoraciones implicitas y lo evaluamos con RMSE, recall@k y map@k, con Numero de Dimensiones Latentes k [10, 30, 50, 100]

In [62]:
for dim in [50]:
    print("*"*30)
    print("Entrenando SVD con %s dimensiones latentes" %dim)
    # Entrenamos con SVD
    u, s, vt = svds(I, dim)
    s_diag_matrix = np.diag(s)
    # calculamos las recomendaciones (predicciones)
    svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD RMSE=%.3f' % rmse(svdPredictions, I2))
    # recall
    svdPredictions_sorted = np.argsort(svdPredictions)[::-1]
    svdPredictions_sorted = svdPredictions_sorted[:, :200]
    test_ids = testUsersGrouped['beer_id'].values
    train_ids = trainUsersGrouped['beer_id'].values
    predicted_ids = trainUsersGrouped['user_id'].apply(lambda i: svdPredictions_sorted[i-1]).values
    topk = [10, 50, 100]
    m = Metrics(train_ids, test_ids, predicted_ids)
    print('*****Recall*****')
    _ = m.recall_at_n(topk)
    print('*****Map*****')
    _ = m.map_at_n(topk)

******************************
Entrenando SVD con 50 dimensiones latentes


MemoryError: 